# Hierarchical Data Structure

* [How netCDF files are organized](https://pro.arcgis.com/en/pro-app/latest/help/data/multidimensional/fundamentals-of-netcdf-data-storage.htm)
* [HDF files](https://docs.h5py.org/en/stable/quick.html)

In order to use the data in this notebook you will need to unzip the three files in this folder.

:::{admonition} Content
:class: note, dropdown

- Background
- HDF4 example
- HDF5 example
- netCDF example

:::

## File Format Structure

### Concept No. 1: pros and cons of lenient file formats

![File Format Spectrum](./images/strict_file_format_spectrum.jpg)

|       | Pro | Con |
| :----------- | :-----------: | :-----------: |
| **Strict File Format**      | easy to know what you're going to get       | doesn't handle all data types       |
| **Not Strict File Format**   | handles lots of data types        | what's inside is unpredictable      |

### Concept No. 2: organizing variables by dimension

<img src="https://pro.arcgis.com/en/pro-app/latest/help/data/multidimensional/GUID-D872A4C3-749E-4159-A6C0-FB6D3B47C5D8-web.gif">

<center>A 3-dimensional dataset</center>

<img src="https://pro.arcgis.com/en/pro-app/latest/help/data/multidimensional/GUID-1D7240CD-54D4-45FF-A150-43B1AFFBF7D6-web.gif">

<center>A 4-dimensional dataset</center>

Images from [Fundamentals of NetCDF Storage](https://pro.arcgis.com/en/pro-app/latest/help/data/multidimensional/fundamentals-of-netcdf-data-storage.htm) by ESRI

When variables are defined in netCDF files they are also assigned dimensions.  **Dimensions** tell us the axis over which our data varies.  Common examples are latitute, longitude, or time.  The values of the dimensions are given in special variables called **coordinate variables**.  The coordinate variables and dimensions help us understand what the core data stored in each variable is describing.  **Attributes** store metadata about our variables.

## Concept No. 3: groups and datasets

In the previous raster lessons we have been using data where the organization is a single dataset per file.  HDF and netCDF are unique in that they allow multiple datasets to be in the same file.  To keep organized, datasets are allowed to be stored together in **groups**.  An analogy is to think of groups like folders in a folder structure and datasets as the individual files.  Groups can have more groups inside of them.

## HDF4

### Install

In Anaconda Powershell:

`conda install -c conda-forge -n lessons pyhdf`

### Documentation

http://fhs.github.io/pyhdf/modules/SD.html  (My opinion: pretty terrible docs)

### Example dataset: CALIPSO

CALIPSO Level 2: `CAL_LID_L2_01kmCLay-Standard-V4-21.2020-07-01T07-32-43ZD.hdf`

[download link](https://drive.google.com/file/d/1AMNZn8u4TGYefZjuo9UjmSXox1LcGdSE/view?usp=sharing)

### Opening the Dataset

In [ ]:
from pyhdf.SD import *

In [ ]:
filepath = './CAL_LID_L2_01kmCLay-Standard-V4-21.2020-07-01T07-32-43ZD.hdf'

In [ ]:
# Opening the file
d = SD(filepath)

In [ ]:
d

### Exploring the dataset

In [ ]:
d.datasets()

Choose a key from the `.datasets()` dictionary and get the varaible info using `.select()`

In [ ]:
d.select('Integrated_Attenuated_Backscatter_532')

### Getting your data

The data can be accessed using `[:]` and is output as a numpy array.  Any of the methods we practiced with numpy arrays in lecture can be used on the dataset.

In [ ]:
d.select('Integrated_Attenuated_Backscatter_532')[:]

In [ ]:
backscatter = d.select('Integrated_Attenuated_Backscatter_532')[:]

In [ ]:
type(backscatter)

In [ ]:
backscatter.shape

In [ ]:
backscatter.max()

### Attributes

Get metadata with the `.attributes()` method.

In [ ]:
d.select('Integrated_Attenuated_Backscatter_532').attributes()

### Masking a no data value

While HDF5 data will automatically mask out nodata values, HDF4 datasets often don't.  To mask them yourself you can look up the fill value and apply it to the array.

In [ ]:
import numpy.ma as ma

In [ ]:
# Mask the array
masked_backscatter = ma.masked_where(backscatter == -9999, backscatter)
# Update the nodata value
ma.set_fill_value(backscatter, -9999)

In [ ]:
masked_backscatter

## HDF5

### Install

In Anaconda Powershell:

`conda install -c conda-forge -n lessons h5py`

### Documentation

https://docs.h5py.org/en/stable/quick.html

### Example Dataset: ASTER Emissivity

`AG100.v003.83.-013.0001.h5`

### Attempting open with xarray

In [ ]:
# Returns empty
xr.open_dataset(filepath)

In [23]:
# Specify group.  If dataset is nested you can do /Emissivity/group2
xr.open_dataset(filepath, group='Emissivity')
# This also works for netCDF

<xarray.Dataset>
Dimensions:  (phony_dim_2: 5, phony_dim_3: 1000, phony_dim_4: 1000)
Dimensions without coordinates: phony_dim_2, phony_dim_3, phony_dim_4
Data variables:
    Mean     (phony_dim_2, phony_dim_3, phony_dim_4) int16 ...
    SDev     (phony_dim_2, phony_dim_3, phony_dim_4) int16 ...

### Opening a Dataset

In [15]:
import h5py

In [16]:
filepath = './AG100.v003.83.-013.0001.h5'

In [17]:
f = h5py.File(filepath, 'r')

In [18]:
f

<HDF5 file "AG100.v003.83.-013.0001.h5" (mode r)>

### Exploring Groups

In [19]:
f.keys()

<KeysViewHDF5 ['ASTER GDEM', 'Emissivity', 'Geolocation', 'Land Water Map', 'NDVI', 'Observations', 'Temperature']>

In [ ]:
f['Emissivity']

In [ ]:
f['Emissivity'].keys()

In [ ]:
f['Emissivity']['Mean']

You can check where you are in the file hierarchy with the `.name` method

In [ ]:
f.name

In [ ]:
f['Emissivity'].name

In [ ]:
f['Emissivity']['Mean'].name

### Getting your data

The data inside the data group dictionaries are numpy arrays, so you can use any of the methods we learned about in other lectures with them.

In [ ]:
mean_emissivity = f['Emissivity']['Mean'][:]

In [ ]:
type(mean_emissivity)

In [ ]:
mean_emissivity.shape

In [ ]:
mean_emissivity.max()

In [ ]:
from matplotlib import pyplot

In [ ]:
pyplot.imshow(mean_emissivity[0])

### Attributes

Metadata in HDF files are called **attributes** and are accessed with `.attrs`

In [ ]:
f['Emissivity']['Mean'].attrs.keys()

In [ ]:
f['Emissivity']['Mean'].attrs['Description']

If there are no attributes for that group you will just get back an empty list

In [ ]:
# No attributes on the Emissivity group
f['Emissivity'].attrs.keys()

In [ ]:
# No attributes on the root group
f.attrs.keys()

## netCDF

### Install

In Anaconda Powershell:

`conda install -c conda-forge -n lessons netcdf4`

### Documentation Link

https://unidata.github.io/netcdf4-python/#creatingopeningclosing-a-netcdf-file

### Example Dataset: MODIS Chlorophyll-a

`A2018006.L3m_DAY_CHL_chlor_a_4km.nc`

### Opening a Dataset - `xarray`

notice comparing to panoply that "processing_control" is missing. Probs because its a group.

In [8]:
import xarray as xr

In [10]:
d_xr = xr.open_dataset(filepath)

d_xr

<xarray.Dataset>
Dimensions:  (lat: 4320, lon: 8640, rgb: 3, eightbitcolor: 256)
Coordinates:
  * lat      (lat) float32 89.98 89.94 89.9 89.85 ... -89.85 -89.9 -89.94 -89.98
  * lon      (lon) float32 -180.0 -179.9 -179.9 -179.9 ... 179.9 179.9 180.0
Dimensions without coordinates: rgb, eightbitcolor
Data variables:
    chlor_a  (lat, lon) float32 ...
    palette  (rgb, eightbitcolor) uint8 ...
Attributes: (12/64)
    product_name:                      A2018006.L3m_DAY_CHL_chlor_a_4km.nc
    instrument:                        MODIS
    title:                             MODISA Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          Aqua
    temporal_range:                    day
    ...                                ...
    identifier_product_doi:            10.5067/AQUA/MODIS/L3M/CHL/2018
    keywords:                          Earth Science > Oceans > Ocean Chemist...
    keywords_vocabulary:               NASA Global Change Master Directory (G...
    data_bins:                         2440618
    data_minimum:                      0.0063775154
    data_maximum:                      98.49251

In [27]:
xr.open_dataset(filepath, group='processing_control/input_parameters')

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes: (12/29)
    par:               A2018006.L3m_DAY_CHL_chlor_a_4km.nc.param
    suite:             CHL
    ifile:             A2018006.L3b_DAY_CHL.nc
    ofile:             A2018006.L3m_DAY_CHL_chlor_a_4km.nc
    oformat:           2
    oformat2:          png
    ...                ...
    product_rgb:       rhos_645,rhos_555,rhos_469
    fudge:             1.0
    threshold:         0
    num_cache:         500
    mask_land:         no
    land:              $OCDATAROOT/common/landmask_GMT15ARC.nc

### Opening a Dataset - `netCDF4`

In [2]:
from netCDF4 import Dataset

In [25]:
filepath = './A2018006.L3m_DAY_CHL_chlor_a_4km.nc'

d = Dataset(filepath, 'r')

Looking at the `Dataset` object gives us a decent overview of our dataset.  We can see metadata applies to the whole dataset. At the very bottom the **dimensions** and **varaiables** are shown, as well as any additional groups.

In [6]:
d

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    product_name: A2018006.L3m_DAY_CHL_chlor_a_4km.nc
    instrument: MODIS
    title: MODISA Level-3 Standard Mapped Image
    project: Ocean Biology Processing Group (NASA/GSFC/OBPG)
    platform: Aqua
    temporal_range: day
    processing_version: 2018.0
    date_created: 2018-03-19T18:42:34.000Z
    history: l3mapgen par=A2018006.L3m_DAY_CHL_chlor_a_4km.nc.param 
    l2_flag_names: ATMFAIL,LAND,HILT,HISATZEN,STRAYLIGHT,CLDICE,COCCOLITH,LOWLW,CHLWARN,CHLFAIL,NAVWARN,MAXAERITER,ATMWARN,HISOLZEN,NAVFAIL,FILTER,HIGLINT
    time_coverage_start: 2018-01-06T00:15:01.000Z
    time_coverage_end: 2018-01-07T02:30:00.000Z
    start_orbit_number: 83382
    end_orbit_number: 83397
    map_projection: Equidistant Cylindrical
    latitude_units: degrees_north
    longitude_units: degrees_east
    northernmost_latitude: 90.0
    southernmost_latitude: -90.0
    westernmost_longitude: -180.0
    easternmost_longi

You can also directly access dimensions or varaiables.  The outputs act like dictionaries, so you can use `.keys()` or `[KEY]` syntax to access elements

In [ ]:
d.dimensions

In [11]:
d.variables

{'chlor_a': <class 'netCDF4._netCDF4.Variable'>
 float32 chlor_a(lat, lon)
     long_name: Chlorophyll Concentration, OCI Algorithm
     units: mg m^-3
     standard_name: mass_concentration_chlorophyll_concentration_in_sea_water
     _FillValue: -32767.0
     valid_min: 0.001
     valid_max: 100.0
     reference: Hu, C., Lee Z., and Franz, B.A. (2012). Chlorophyll-a algorithms for oligotrophic oceans: A novel approach based on three-band reflectance difference, J. Geophys. Res., 117, C01011, doi:10.1029/2011JC007395.
     display_scale: log
     display_min: 0.01
     display_max: 20.0
 unlimited dimensions: 
 current shape = (4320, 8640)
 filling on,
 'lat': <class 'netCDF4._netCDF4.Variable'>
 float32 lat(lat)
     long_name: Latitude
     units: degrees_north
     standard_name: latitude
     _FillValue: -999.0
     valid_min: -90.0
     valid_max: 90.0
 unlimited dimensions: 
 current shape = (4320,)
 filling on,
 'lon': <class 'netCDF4._netCDF4.Variable'>
 float32 lon(lon)
     l

In [12]:
# Treating a variable like a dictionary
print(d.variables.keys())
d.variables['chlor_a']

dict_keys(['chlor_a', 'lat', 'lon', 'palette'])


<class 'netCDF4._netCDF4.Variable'>
float32 chlor_a(lat, lon)
    long_name: Chlorophyll Concentration, OCI Algorithm
    units: mg m^-3
    standard_name: mass_concentration_chlorophyll_concentration_in_sea_water
    _FillValue: -32767.0
    valid_min: 0.001
    valid_max: 100.0
    reference: Hu, C., Lee Z., and Franz, B.A. (2012). Chlorophyll-a algorithms for oligotrophic oceans: A novel approach based on three-band reflectance difference, J. Geophys. Res., 117, C01011, doi:10.1029/2011JC007395.
    display_scale: log
    display_min: 0.01
    display_max: 20.0
unlimited dimensions: 
current shape = (4320, 8640)
filling on

In [ ]:
# Access metadata with using a period .
d.variables['chlor_a'].long_name

If the data you need is in a group, it can be accessed with `.groups`

In [13]:
d.groups['processing_control']

<class 'netCDF4._netCDF4.Group'>
group /processing_control:
    software_name: l3mapgen
    software_version: 2.0.0-V2018.0.6
    source: A2018006.L3b_DAY_CHL.nc
    l2_flag_names: ATMFAIL,LAND,HILT,HISATZEN,STRAYLIGHT,CLDICE,COCCOLITH,LOWLW,CHLWARN,CHLFAIL,NAVWARN,MAXAERITER,ATMWARN,HISOLZEN,NAVFAIL,FILTER,HIGLINT
    dimensions(sizes): 
    variables(dimensions): 
    groups: input_parameters

### Getting your data

Once you traverse the file structure with the right keys and find a `netCDF4._netCDF4.Variable` object you can accesss pull out the numpy array and work with it.

In [ ]:
chlor_a = d.variables['chlor_a'][:]

In [ ]:
type(chlor_a)

In [ ]:
chlor_a.shape

In [ ]:
chlor_a.max()

In [ ]:
from matplotlib import pyplot

In [ ]:
pyplot.imshow(chlor_a)

## Raster Data Structure